**Table of contents**<a id='toc0_'></a>    
- [Imports](#toc1_1_)    
- [Help functions](#toc2_)    
- [Модель успешного абитуриента 2022-2023. Данные за весь год.](#toc3_)    
  - [Задача](#toc3_1_)    
  - [Load](#toc3_2_)    
  - [Transform](#toc3_3_)    
  - [Preprocessing](#toc3_4_)    
    - [Дополнительные признаки](#toc3_4_1_)    
    - [Заполнение пропусков (imputer)](#toc3_4_2_)    
    - [Поменяем тип для категорной переменной `grade`](#toc3_4_3_)    
  - [Model](#toc3_5_)    
  - [Baseline](#toc3_6_)    
    - [2021-2022](#toc3_6_1_)    
    - [2022-2023](#toc3_6_2_)    
    - [Все года](#toc3_6_3_)    
  - [Tuning](#toc3_7_)    
  - [Предсказание для 5-7 классов](#toc3_8_)    
  - [Граница скоринга для 2023-2024 уч. года](#toc3_9_)    
  - [Вывод](#toc3_10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Imports](#toc0_)

In [450]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import random

from typing import Optional, Union

from catboost import CatBoostClassifier, cv, Pool

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

from optuna import Trial, create_study
from optuna.visualization import plot_param_importances, plot_optimization_history

from joblib import dump, load

LETOVO_PALETTE = ["#0D3174", "#FDC300", "#A40C30", "#00ADB9", "#B2B2B2"]
sns.set_style("darkgrid")
sns.set_palette("Set2")

plt.rcParams["figure.figsize"] = (16, 10)
plt.rcParams["font.size"] = 20

YELLOW = "#FDC300"
BLUE = "#0D3174"
palette_dict = {1: YELLOW, 0: "grey"}

import warnings

warnings.filterwarnings("ignore")

YEARS = [22, 23]
PATH = "../../Выгрузки/"
OPTUNA_N_TRIALS = 100

# <a id='toc2_'></a>[Help functions](#toc0_)

# <a id='toc3_'></a>[Модель успешного абитуриента 2022-2023. Данные за весь год.](#toc0_)

## <a id='toc3_1_'></a>[Задача](#toc0_)

Задача по результатам занятий на ЛО предсказать, что ученик будет рекомендован к поступлению в школу Летово.

Еще точнее нам нужно узнать какие из текущих учеников 5 класса похожи на рекомендованных 6 класса, и так далее для 6 и 7 классов.

__О наших данных__

- Срез данных делается по 01.03.2023 года
- Тесты берем по трем основным предметам (Рус, Англ, Мат) и только базовой сложности 

__Поля__

Признаки успешности абитуриента 

(во всех полях с тестами, курсами и т.д. NULL - означает, что не присупал)

- `old_lo_id` - старый ЛО ID (в схеме letovo_do_2020, а также lo_20XX)
- `lo_id` - новый ЛО ID (в схемах letovo_online) 
- `crm_lead_id` - старый ID ЕЛК
- `first_attempt_avg_score` - средняя оценка за начальные тесты
- `train_count_avg` - кол-во тренировочных попыток в тесте (среднее по всем предметам).
- `final_score_avg` - средняя финальная оценка за тесты
- `train_count_80_avg` - кол-во тренировочных попыток для достижения 80% в тесте (среднее по всем предметам). 
- `oz_done_count` - кол-во решенных ОЗ (с ненулевым баллом)
- `oz_subject_count` - кол-во различных предметов, в которых были набраны баллы ОЗ
- `oz_percent_avg` - средний балл за ОЗ в процентах от максимального балла за задание
- `course_start` - кол-во курсов, в которых отправлено хотя бы одно задание
- `course_50` - кол-во курсов, в которых отправлено хотя бы половина заданий
- `olymp_start` - кол-во олимпиад, в которых отправлено хотя бы одно задание
- `wave_login_first` - уч.год первого логина, 0 - текущий, 1 - предыдущий и т.д.
- `week_first_login_wave` - порядковый номер недели учебного года, когда был первый логин в год первого логина
- `week_first_login_this_year` - порядковый номер недели учебного года, когда был первый логин в этом году
- `grade` - класс ученика в данном учебном году

## <a id='toc3_2_'></a>[Load](#toc0_)

Загружаем признаки для 5-8 класса

In [221]:
feat = {}

In [368]:
feat[22] = pd.read_csv(PATH + "success_features_2021_2022_up_to_03_01.csv")
feat[23] = pd.read_csv(PATH + "success_features_2022_2023_up_to_03_01.csv")
display(feat[22])
feat[23]

,lo_id,old_lo_id,crm_lead_id,first_attempt_avg_score,train_count_avg,final_score_avg,train_count_80_avg,oz_done_count,oz_subject_count,oz_percent_avg,course_start,course_50,olymp_start,week_first_login_this_year,week_first_login_wave,wave_login_first,grade
0,380726,9803,1995,66.0,23.0,100.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2,18,2,7
1,380802,9881,36617,64.0,0.0,64.0,NaN,8.0,1.0,50.000000,NaN,NaN,NaN,1,3,2,8
2,380803,9882,713,100.0,0.0,100.0,0.0,1.0,1.0,33.333333,NaN,NaN,NaN,19,17,2,7
3,380867,9946,36638,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,NaN,NaN,1,11,2,7
4,380908,9987,1672,26.5,0.0,26.5,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,16,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24040,599343,222985,225822,13.5,0.0,13.5,NaN,0.0,0.0,0.000000,NaN,NaN,NaN,26,26,0,7
24041,599345,223038,225875,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,26,0,5
24042,599346,223042,225879,9.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,26,0,7
24043,599348,223133,225969,0.0,0.0,0.0,NaN,0.0,0.0,0.000000,NaN,NaN,NaN,26,26,0,5


,lo_id,first_attempt_avg_score,train_count_avg,final_score_avg,train_count_80_avg,oz_done_count,oz_subject_count,oz_percent_avg,course_start,course_50,olymp_start,week_first_login_this_year,wave_login_first,week_first_login_wave,grade
0,528789,76.631700,1.6667,92.657333,0.6667,NaN,NaN,NaN,NaN,NaN,1.0,0,1,2,6
1,587048,NaN,NaN,NaN,NaN,4.0,1.0,80.000000,NaN,NaN,NaN,0,1,23,8
2,593642,NaN,0.0000,NaN,NaN,12.0,2.0,92.307692,NaN,NaN,NaN,0,1,37,7
3,601156,66.259233,1.3333,74.051433,1.0000,14.0,1.0,53.846154,7.0,4.0,1.0,0,1,41,8
4,527197,52.840900,9.5000,52.840900,0.0000,1.0,1.0,100.000000,8.0,6.0,NaN,0,2,42,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12617,657926,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,NaN,NaN,26,0,26,6
12618,657935,12.500000,0.0000,12.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,0,26,6
12619,657937,48.701300,1.0000,48.701300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,0,26,8
12620,648331,21.428600,0.0000,21.428600,NaN,5.0,2.0,50.000000,NaN,NaN,NaN,26,0,26,8


In [369]:
feat[22].grade.value_counts()

grade
7    6738
6    6582
8    6531
5    4194
Name: count, dtype: int64

In [370]:
feat[23].grade.value_counts()

grade
8    4167
7    3619
6    3583
5    1253
Name: count, dtype: int64

Загружаем данные о рекомендации:

In [225]:
feat_names = feat[23].columns[1:].tolist()
feat_names

['first_attempt_avg_score',
 'train_count_avg',
 'final_score_avg',
 'train_count_80_avg',
 'oz_done_count',
 'oz_subject_count',
 'oz_percent_avg',
 'course_start',
 'course_50',
 'olymp_start',
 'week_first_login_this_year',
 'wave_login_first',
 'week_first_login_wave',
 'grade']

In [226]:
ru_names = [
    "Результат нач. попытки, %, сред.",
    "Кол-во тренировок, сред.",
    "Финальный результат, %, сред.",
    "Кол-во тренировок до 80%, сред.",
    "Кол-во решенных ОЗ",
    "Кол-во предметов в ОЗ",
    "Средний результат ОЗ, %",
    "Кол-во начатых курсов",
    "Кол-во курсов, пройденных хотя бы на половину",
    "Кол-во начатых олимп.",
    "Неделя первого логина в тек. уч. году",
    "Уч. год первого логина",
    "Неделя первого логина",
    "Класс",
]

ru_dict = dict(zip(feat_names, ru_names))
display(len(ru_dict))
ru_dict

14

{'first_attempt_avg_score': 'Результат нач. попытки, %, сред.',
 'train_count_avg': 'Кол-во тренировок, сред.',
 'final_score_avg': 'Финальный результат, %, сред.',
 'train_count_80_avg': 'Кол-во тренировок до 80%, сред.',
 'oz_done_count': 'Кол-во решенных ОЗ',
 'oz_subject_count': 'Кол-во предметов в ОЗ',
 'oz_percent_avg': 'Средний результат ОЗ, %',
 'course_start': 'Кол-во начатых курсов',
 'course_50': 'Кол-во курсов, пройденных хотя бы на половину',
 'olymp_start': 'Кол-во начатых олимп.',
 'week_first_login_this_year': 'Неделя первого логина в тек. уч. году',
 'wave_login_first': 'Уч. год первого логина',
 'week_first_login_wave': 'Неделя первого логина',
 'grade': 'Класс'}

Ученики, подавшие заявку в школу в 2021-2022, а также рекомендованные к поступлению

In [227]:
claims = {}

Для текущего года у нас уже есть готовая выгрузка

In [228]:
claims[23] = pd.read_csv(
    PATH + "claims_recomend_2022_2023.csv", parse_dates=["submitted_at"]
)
claims[23] = claims[23].drop(columns="grade")
claims[23]

,lo_id,submitted_at,recomended
0,538975,2022-09-30 22:12:40,0
1,499683,2022-09-28 11:08:42,0
2,600915,2022-09-29 22:10:20,0
3,500082,2022-09-01 07:18:02,0
4,505971,2022-09-18 11:54:59,1
...,...,...,...
4589,625282,2022-10-28 06:57:24,0
4590,625271,2022-10-28 09:11:40,0
4591,625150,2022-10-28 03:13:42,0
4592,625146,2022-10-27 21:34:54,0


Для прошлого года придется ее собрать по кусочкам

In [229]:
claims[22] = pd.read_csv("claims_2021_2022.csv")
claims[22]

,crm_lead_id,recomended
0,3907,0
1,3302,0
2,1158,0
3,3745,0
4,3617,0
...,...,...
6895,96622,0
6896,231901,0
6897,176019,0
6898,44837,0


In [230]:
recom_cnt = {}

for i in YEARS:
    recom_cnt[i] = (claims[i].shape[0], claims[i]["recomended"].sum())

claim_cnt = pd.DataFrame(recom_cnt, index=["Заявки", "Рекомендовано"])
claim_cnt

,22,23
Заявки,6900,4594
Рекомендовано,438,252


## <a id='toc3_3_'></a>[Transform](#toc0_)

В этом разделе подготовим данные для моделирования в формате удобном для sklearn, т.е. `numpy.array`

In [231]:
data = {}

In [232]:
data[22] = feat[22].merge(claims[22], on="crm_lead_id")
data[22] = data[22][feat_names], data[22]["recomended"]
data[22]

(      first_attempt_avg_score  train_count_avg  final_score_avg  \
 0                    100.0000              0.0         100.0000   
 1                         NaN              NaN              NaN   
 2                     26.5000              0.0          26.5000   
 3                     23.6667              8.0          90.3333   
 4                     74.6667              2.0         100.0000   
 ...                       ...              ...              ...   
 5154                   0.0000              0.0           0.0000   
 5155                  41.0000              0.0          41.0000   
 5156                   0.0000              0.0           0.0000   
 5157                  44.3333              3.0          81.3333   
 5158                   0.0000              0.0           0.0000   
 
       train_count_80_avg  oz_done_count  oz_subject_count  oz_percent_avg  \
 0                 0.0000            1.0               1.0       33.333333   
 1                    NaN 

In [233]:
data[23] = feat[23].merge(claims[23], on="lo_id")
data[23] = data[23][feat_names], data[23]["recomended"]
data[23]

(      first_attempt_avg_score  train_count_avg  final_score_avg  \
 0                     76.3333           1.6667          14.0000   
 1                     52.5000           9.5000           4.5000   
 2                     40.3333           0.6667           5.3333   
 3                     22.0000           3.6667          17.0000   
 4                     76.0000           6.6667          14.6667   
 ...                       ...              ...              ...   
 3395                  46.0000           1.0000           5.0000   
 3396                   0.0000           0.0000              NaN   
 3397                  32.0000           0.3333           8.0000   
 3398                   0.0000           0.0000              NaN   
 3399                  21.0000           0.0000           3.0000   
 
       train_count_80_avg  oz_done_count  oz_subject_count  oz_percent_avg  \
 0                 0.6667            NaN               NaN             NaN   
 1                 0.0000 

Также попробуем объединить данные

In [239]:
pd.concat([data[22][0], data[23][0]], axis=0)

,first_attempt_avg_score,train_count_avg,final_score_avg,train_count_80_avg,oz_done_count,oz_subject_count,oz_percent_avg,course_start,course_50,olymp_start,week_first_login_this_year,wave_login_first,week_first_login_wave,grade
0,100.0000,0.0000,100.0000,0.0000,1.0,1.0,33.333333,NaN,NaN,NaN,19,2,17,7
1,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,NaN,NaN,1,2,11,7
2,26.5000,0.0000,26.5000,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,2,16,8
3,23.6667,8.0000,90.3333,4.0000,0.0,0.0,0.000000,NaN,NaN,NaN,2,2,3,8
4,74.6667,2.0000,100.0000,0.6667,29.0,3.0,50.892857,NaN,NaN,NaN,21,0,21,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,46.0000,1.0000,5.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,0,26,7
3396,0.0000,0.0000,NaN,NaN,0.0,NaN,0.000000,NaN,NaN,NaN,26,0,26,8
3397,32.0000,0.3333,8.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,0,26,6
3398,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,0,26,8


In [279]:
data_all = pd.concat([data[22][0], data[23][0]], axis=0).reset_index(
    drop=True
), pd.concat([data[22][1], data[23][1]], axis=0).reset_index(drop=True)
index = data_all[1].index.tolist()
random.Random(24).shuffle(index)
data_all = data_all[0].loc[index, :].reset_index(drop=True), data_all[1].loc[
    index
].reset_index(drop=True)
data_all

(      first_attempt_avg_score  train_count_avg  final_score_avg  \
 0                     26.3333              4.0          28.6667   
 1                     72.0000              0.0           7.5000   
 2                     40.0000              0.0          40.0000   
 3                     83.6667              1.0          88.0000   
 4                      0.0000              0.0              NaN   
 ...                       ...              ...              ...   
 8554                  57.0000             26.0         100.0000   
 8555                  41.6667              0.0          41.6667   
 8556                  55.0000             10.0         100.0000   
 8557                  54.6667              0.0          36.5000   
 8558                  74.0000             10.0          14.0000   
 
       train_count_80_avg  oz_done_count  oz_subject_count  oz_percent_avg  \
 0                    NaN            2.0               1.0       66.666667   
 1                 0.0000 

## <a id='toc3_4_'></a>[Preprocessing](#toc0_)

### <a id='toc3_4_1_'></a>[Дополнительные признаки](#toc0_)

In [242]:
def count_na_features(df):
    "Counts na features in each row"
    df["na_feature_cnt"] = df.isna().sum(axis=1)
    return df


count_na_features(data[22][0]).head()

,first_attempt_avg_score,train_count_avg,final_score_avg,train_count_80_avg,oz_done_count,oz_subject_count,oz_percent_avg,course_start,course_50,olymp_start,week_first_login_this_year,wave_login_first,week_first_login_wave,grade,na_feature_cnt
0,100.0000,0.0,100.0000,0.0000,1.0,1.0,33.333333,NaN,NaN,NaN,19,2,17,7,3
1,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,NaN,NaN,1,2,11,7,7
2,26.5000,0.0,26.5000,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,2,16,8,6
3,23.6667,8.0,90.3333,4.0000,0.0,0.0,0.000000,NaN,NaN,NaN,2,2,3,8,3
4,74.6667,2.0,100.0000,0.6667,29.0,3.0,50.892857,NaN,NaN,NaN,21,0,21,6,3


In [243]:
feature_maker = FunctionTransformer(count_na_features)

In [244]:
feat_names.append("na_feature_cnt")
ru_dict["na_feature_cnt"] = "Кол-во отсутствующих признаков"

### <a id='toc3_4_2_'></a>[Заполнение пропусков (imputer)](#toc0_)

В этом подразделе разберемся с пропущенными данными:

In [442]:
feat_names

['first_attempt_avg_score',
 'train_count_avg',
 'final_score_avg',
 'train_count_80_avg',
 'oz_done_count',
 'oz_subject_count',
 'oz_percent_avg',
 'course_start',
 'course_50',
 'olymp_start',
 'week_first_login_this_year',
 'wave_login_first',
 'week_first_login_wave',
 'grade']

In [245]:
# fill_with_999 = ['train_count_avg', 'train_count_80_avg']

fill_missing = ColumnTransformer(
    [
        # тренировочные попытки заполняем 999
        (
            "fill_missing",
            SimpleImputer(strategy="constant", fill_value=999),
            make_column_selector(pattern="train"),
        )
    ],
    # остальные значения заполняем -1
    remainder=SimpleImputer(strategy="constant", fill_value=-1),
    # не меняем названия признаков
    verbose_feature_names_out=False,
)

### <a id='toc3_4_3_'></a>[Поменяем тип для категорной переменной `grade`](#toc0_)

In [246]:
def change_type(df):
    try:
        df["grade"] = df["grade"].astype("int")
    except:
        return df
    return df


change_type_transformer = FunctionTransformer(change_type)

In [247]:
X, y = data[22]
preprocessor = Pipeline(
    steps=[
        ("na_count", feature_maker),
        ("imputer", fill_missing),
        ("type_change", change_type_transformer)
        # ('feature_selection', SelectFromModel(CatBoostClassifier(n_estimators=500, random_state=0, cat_features=['grade'], verbose=False), threshold='0.5*median')),
    ]
).set_output(transform="pandas")

X = preprocessor.fit_transform(X, y)
print(np.isnan(X).sum())
print(X.info())
X[:5]

train_count_avg               0
train_count_80_avg            0
first_attempt_avg_score       0
final_score_avg               0
oz_done_count                 0
oz_subject_count              0
oz_percent_avg                0
course_start                  0
course_50                     0
olymp_start                   0
week_first_login_this_year    0
wave_login_first              0
week_first_login_wave         0
grade                         0
na_feature_cnt                0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5159 entries, 0 to 5158
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   train_count_avg             5159 non-null   float64
 1   train_count_80_avg          5159 non-null   float64
 2   first_attempt_avg_score     5159 non-null   float64
 3   final_score_avg             5159 non-null   float64
 4   oz_done_count               5159 non-null   float64
 

,train_count_avg,train_count_80_avg,first_attempt_avg_score,final_score_avg,oz_done_count,oz_subject_count,oz_percent_avg,course_start,course_50,olymp_start,week_first_login_this_year,wave_login_first,week_first_login_wave,grade,na_feature_cnt
0,0.0,0.0000,100.0000,100.0000,1.0,1.0,33.333333,-1.0,-1.0,-1.0,19.0,2.0,17.0,7,3.0
1,999.0,999.0000,-1.0000,-1.0000,0.0,0.0,0.000000,-1.0,-1.0,-1.0,1.0,2.0,11.0,7,7.0
2,0.0,999.0000,26.5000,26.5000,-1.0,-1.0,-1.000000,-1.0,-1.0,1.0,1.0,2.0,16.0,8,6.0
3,8.0,4.0000,23.6667,90.3333,0.0,0.0,0.000000,-1.0,-1.0,-1.0,2.0,2.0,3.0,8,3.0
4,2.0,0.6667,74.6667,100.0000,29.0,3.0,50.892857,-1.0,-1.0,-1.0,21.0,0.0,21.0,6,3.0


## <a id='toc3_5_'></a>[Model](#toc0_)

Кастомизируем стандартный catboost, чтобы после скоринга метод `predict` возвращал топ 10%, как успех. 

In [248]:
def predict_with_cutoff(y_prob, percent=10, return_threshold=False):
    # print(event_rate)
    size = int(round(y_prob.shape[0] * percent / 100, 0))
    indxs = np.argpartition(y_prob, -size)[-size:]

    # print ("Cutoff/threshold at: " + str(threshold))
    # print(y_prob[np.min(indxs)])
    y_pred = np.zeros(y_prob.shape)
    y_pred[indxs] = 1

    if return_threshold:
        threshold = np.min(y_prob[indxs])

        return y_pred, threshold

    return y_pred


y_prob = np.arange(20) / 20
display(y_prob)
predict_with_cutoff(y_prob, 10)

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1.])

In [249]:
predict_with_cutoff(y_prob, 10, True)

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1.]),
 0.9)

In [250]:
class MyCatboostClassifier(CatBoostClassifier):
    def predict(self, X, percent=10, **kwargs):
        result = super(MyCatboostClassifier, self).predict_proba(X, **kwargs)[:, 1]
        predictions = predict_with_cutoff(result, percent)
        return predictions

In [258]:
def cv_model(
    model, X, y, scoring=["precision", "recall", "f1"], cv=3, feat_names=feat_names
):
    cv_result = cross_validate(
        model, X, y, cv=cv, scoring=scoring, return_estimator=True, n_jobs=-1
    )

    scores_dict = {}
    for scorer in scoring:
        scores_dict[scorer] = cv_result["test_" + scorer]

    cv_result_df = pd.DataFrame(scores_dict)
    cv_result_df = cv_result_df.apply(["mean", "std"], axis=0).round(2)
    print("Test scores:")
    display(cv_result_df)

    feat_importance_dict = {}
    for idx, estimator in enumerate(cv_result["estimator"]):
        try:
            feat_importance_dict[idx] = estimator[1].feature_importances_
        except:
            feat_importance_dict[idx] = estimator.feature_importances_

    feat_importance = (
        pd.DataFrame(
            feat_importance_dict,
            index=[ru_dict[key] for key in feat_names if key in ru_dict],
        )
        .apply(["mean", "std"], axis=1)
        .sort_values("mean", ascending=False)
        .round(2)
    )
    print("Feature importances:")
    display(feat_importance)

    return cv_result

## <a id='toc3_6_'></a>[Baseline](#toc0_)

Получим базовое значений метрик recall, precision от которого сможем отталкиваться

### <a id='toc3_6_1_'></a>[2021-2022](#toc0_)

In [134]:
X, y = data[22]

In [208]:
cat_features = ["grade"]

params = {
    "eval_metric": "Recall",
    "cat_features": cat_features,
    "verbose": False,
    "random_state": 0,
}

cbc = MyCatboostClassifier(**params)
pipe = Pipeline(steps=[("preprocessor", preprocessor), ("catboost", cbc)])

cv_result = cv_model(pipe, X, y)

# baseline_scores = cross_val_score(pipe, X, y, scoring="recall", cv=3, n_jobs=-1)
# print(f"Базовый Recall = {baseline_scores.mean():.2%} +- {baseline_scores.std():.2%}")

Test scores:


,precision,recall,f1
mean,0.30,0.36,0.33
std,0.03,0.04,0.03


Feature importances:


,mean,std
"Финальный результат, %, сред.",12.91,0.50
Класс,11.88,0.87
"Кол-во тренировок, сред.",11.63,2.02
"Кол-во тренировок до 80%, сред.",11.52,1.08
Неделя первого логина в тек. уч. году,11.50,2.88
Неделя первого логина,8.72,0.92
"Средний результат ОЗ, %",7.42,0.85
"Результат нач. попытки, %, сред.",6.37,1.02
Уч. год первого логина,5.75,0.91
Кол-во решенных ОЗ,5.47,0.43


Мы видим, что наиболее ценными оказались тесты, особенно __финальный результат__, а также __кол-во тренировок. Это было ожидаемо, ведь большинство рекомендованных решили 3 теста хотя бы на 80%. 

__Класс__ - также понятный признак, поскольку результаты довольно сильно разнятся от класса к классу.

__Неделя первого логина в текущем году__ - более неожиданный признак, видимо большниство рекомендованных логинилось в начале года.

Олимпиадные задания также дают немалый вклад в предсказание, а вот курсы и олимпиады практически не имеют значения. Созданный нами признак считающий кол-во отсутствующих признаков также не слишком интересен.

### <a id='toc3_6_2_'></a>[2022-2023](#toc0_)

Посмотрим на то же самое в 2022-2023 году

In [215]:
X, y = data[23]
cv_model(pipe, X, y)

Test scores:


,precision,recall,f1
mean,0.16,0.23,0.19
std,0.17,0.25,0.20


Feature importances:


,mean,std
Неделя первого логина в тек. уч. году,22.67,6.09
"Финальный результат, %, сред.",12.43,0.78
"Кол-во тренировок, сред.",8.89,1.75
Неделя первого логина,8.78,0.66
"Кол-во тренировок до 80%, сред.",8.33,1.80
"Результат нач. попытки, %, сред.",7.81,0.65
Класс,6.08,2.66
"Средний результат ОЗ, %",4.56,0.60
Кол-во решенных ОЗ,3.87,0.64
Кол-во отсутствующих признаков,3.84,1.02


Тут мы видим, что огромное значение модель при обучении придает __неделе первого логина в текущем году__. Т.е. видимо в данном случае, еще сильнее разница во времени логина рекомедованных и всех остальных.

Курсы, олимпиады и кол-во отсутствующих признаков по-прежнему в хвосте, но к ним добавился также кругозор ОЗ.

Также нужно отметить, что результаты скоринга гораздо хуже. Вероятно это связанно с меньшим кол-во данных, все-таки разница почти в 1,5 раза.

### <a id='toc3_6_3_'></a>[Все года](#toc0_)

Посмотрим на объединенные данные, тут можно взять побольше fold-ов

In [284]:
X, y = data_all

In [285]:
cv_model(pipe, X, y, cv=6)

Test scores:


,precision,recall,f1
mean,0.29,0.37,0.33
std,0.02,0.03,0.03


Feature importances:


,mean,std
"Финальный результат, %, сред.",15.59,0.85
"Кол-во тренировок, сред.",11.89,1.09
Неделя первого логина в тек. уч. году,9.64,0.97
"Кол-во тренировок до 80%, сред.",9.60,0.96
"Результат нач. попытки, %, сред.",8.99,0.49
Неделя первого логина,8.06,0.62
"Средний результат ОЗ, %",6.96,0.62
Класс,5.59,1.15
Кол-во отсутствующих признаков,5.57,0.97
Кол-во решенных ОЗ,5.41,0.15


Получили нечто среднее между двумя годами, но ближе к 2021-2022. Это объясняется тем, что данных там побольше.

Мне нравится идея обучаться на нескольких годах, так мы еще регуляризируем модель, а также даем ей побольше данных.

Попробуем выкинуть самые слабые признаки, связанные с курсами и олимпиадами:

In [286]:
X = X.drop(columns=["course_start", "course_50", "olymp_start"])
cv_model(pipe, X, y, feat_names=feat_names[:7] + feat_names[10:], cv=6)

Test scores:


,precision,recall,f1
mean,0.27,0.35,0.30
std,0.02,0.03,0.03


Feature importances:


,mean,std
"Финальный результат, %, сред.",16.63,0.97
"Кол-во тренировок, сред.",12.81,0.83
Неделя первого логина в тек. уч. году,10.45,0.40
"Кол-во тренировок до 80%, сред.",9.99,1.02
"Результат нач. попытки, %, сред.",9.84,0.72
Неделя первого логина,8.65,0.66
"Средний результат ОЗ, %",7.28,0.71
Класс,6.46,1.49
Кол-во решенных ОЗ,5.78,0.18
Уч. год первого логина,4.73,0.45


Мы видим, что в принципе стало чуть хуже, поэтому оставим все имеющиеся признаки и перейдем к тюнингу

## <a id='toc3_7_'></a>[Tuning](#toc0_)

В этом разделе определим применим optuna для тюнинга гиперпараметров модели и препроцессинга. Сам тюнинг лучше осуществлять на серверах Kaggle или чем-то аналогичном

Идея писать instantiate фунцкии для optuna взята [отсюда](https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451)

In [287]:
def instantiate_processor(trial: Trial):
    return preprocessor

In [288]:
def instantiate_сatboost(trial: Trial) -> MyCatboostClassifier:
    params = {
        "iterations": trial.suggest_categorical(
            "iterations", [125, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]
        ),
        "learning_rate": trial.suggest_categorical(
            "learning_rate", [0.1, 0.05, 0.025, 0.005, 0.0025, 0.001]
        ),
        "depth": trial.suggest_int("depth", 1, 10),
        "random_strength": trial.suggest_float("random_strength", 0.0, 5.0),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.0, 6.0),
        "early_stopping_rounds": trial.suggest_categorical(
            "early_stopping_rounds", [False, 15, 30]
        ),
        "random_state": 0,
        "verbose": False,
    }

    cbc = MyCatboostClassifier(**params)
    return cbc

In [289]:
def instantiate_model(trial: Trial) -> Pipeline:
    processor = instantiate_processor(trial)

    cbc = instantiate_сatboost(trial)

    model = Pipeline([("processor", processor), ("catboost", cbc)])

    return model

In [291]:
def objective(trial: Trial, X: pd.DataFrame, y: Union[pd.Series, np.ndarray]) -> float:
    model = instantiate_model(trial)

    scores = cross_val_score(model, X, y, scoring="recall", cv=6, n_jobs=-1)

    return np.min([np.mean(scores), np.median([scores])])

In [292]:
X, y = data_all

study = create_study(study_name="optimization", direction="maximize")
study.optimize(lambda trial: objective(trial, X, y), n_trials=OPTUNA_N_TRIALS)

[I 2023-07-20 13:10:47,860] A new study created in memory with name: optimization
[I 2023-07-20 13:11:36,329] Trial 0 finished with value: 0.34848484848484845 and parameters: {'iterations': 1500, 'learning_rate': 0.005, 'depth': 2, 'random_strength': 0.43047713027623724, 'l2_leaf_reg': 2.77386690580186, 'early_stopping_rounds': 30}. Best is trial 0 with value: 0.34848484848484845.
[I 2023-07-20 13:12:27,505] Trial 1 finished with value: 0.34848484848484845 and parameters: {'iterations': 1750, 'learning_rate': 0.1, 'depth': 2, 'random_strength': 1.2812550116923709, 'l2_leaf_reg': 4.7414039050052175, 'early_stopping_rounds': 30}. Best is trial 0 with value: 0.34848484848484845.


In [293]:
fig = plot_optimization_history(study)
display(fig)

try:
    fig = plot_param_importances(study)
    display(fig)
except RuntimeError:
    1 == 1

best_params = study.best_params
display(best_params)

best_model = instantiate_model(study.best_trial)
best_model.save_model("best_tuned_model")
cv_model(best_model, X, y)

{'iterations': 1500,
 'learning_rate': 0.005,
 'depth': 2,
 'random_strength': 0.43047713027623724,
 'l2_leaf_reg': 2.77386690580186,
 'early_stopping_rounds': 30}

Test scores:


,precision,recall,f1
mean,0.30,0.39,0.34
std,0.01,0.01,0.01


ValueError: Length of values (12) does not match length of index (15)

## Application

Загрузим уже оттюнингованную модель

In [296]:
cbc_tune = MyCatboostClassifier()
cbc_tune.load_model("best_tuned_model")
cbc_tune.get_params()

{'random_strength': 1.441532743,
 'od_wait': 15,
 'verbose': 0,
 'iterations': 250,
 'loss_function': 'Logloss',
 'l2_leaf_reg': 2.647452459,
 'depth': 10,
 'random_seed': 0,
 'learning_rate': 0.001}

Удалим данные о рекомендованных учениках 2022-2023, включая Fast Track

In [302]:
recomend_ids = [
    575488,
    639873,
    540299,
    599996,
    618406,
    394781,
    529895,
    566883,
    602600,
    608726,
    612199,
    569453,
    566371,
    527315,
    496892,
    634017,
    586658,
    386193,
    383221,
    569684,
    583835,
    666766,
    559362,
    610794,
    549630,
    600954,
    536052,
    583106,
    530096,
    549402,
    609095,
    605196,
    649599,
    543039,
    593506,
    610203,
    495529,
    614072,
    651108,
    604868,
    431793,
    619258,
    529866,
    592482,
    593603,
    620648,
    614295,
    475203,
    529068,
    621002,
    615731,
    558299,
    527012,
    529229,
    529290,
    605470,
    527027,
    610721,
    547585,
    528437,
    632960,
    451060,
    571874,
    566699,
    614645,
    430940,
    546054,
    612323,
    623788,
    614822,
    527275,
    564364,
    653659,
    647126,
    639058,
    495508,
    613032,
    605464,
    518622,
    603432,
    565356,
    567003,
    633372,
    436886,
    654681,
    614833,
    549853,
    557949,
    662751,
    566540,
    608821,
    604993,
    503938,
    619316,
    618340,
    650847,
    614476,
    453566,
    609797,
    462133,
    638946,
    605442,
    482247,
    619570,
    574231,
    601926,
    616912,
    565915,
    614408,
    608340,
    505528,
    655582,
    509535,
    608290,
    568114,
    502058,
    502866,
    629068,
    545491,
    557913,
    422534,
    504586,
    491714,
    636284,
    570093,
    534858,
    529533,
    548838,
    620074,
    529970,
    635709,
    605935,
    608915,
    528951,
    641921,
    587384,
    648001,
    566273,
    604367,
    539524,
    529035,
    649180,
    574675,
    650622,
    382951,
    569259,
    522998,
    520365,
    654453,
    654288,
    610907,
    500653,
    529413,
    566958,
    624197,
    607633,
    630751,
    532384,
    565330,
    588808,
    533128,
    587373,
    384839,
    640452,
    430166,
    529562,
    637428,
    556891,
    527242,
    524969,
    567613,
    549949,
    570009,
    568976,
    656526,
    585464,
    533084,
    505971,
    571023,
    502382,
    519967,
    618127,
    602450,
    529173,
    467081,
    495168,
    527313,
    500184,
    551380,
    530324,
    572359,
    612097,
    553615,
    629640,
    646602,
    457248,
    608818,
    512237,
    387007,
    650502,
    597343,
    562998,
    480045,
    623964,
    482263,
    547283,
    576086,
    651121,
    552465,
    657636,
    583104,
    608863,
    420603,
    586589,
    559064,
    593076,
    572536,
    616975,
    608547,
    544593,
    551727,
    555246,
    488473,
    613365,
    620050,
    607983,
    527683,
    567811,
    651290,
    520946,
    497032,
    542886,
    615309,
    628184,
    522955,
    464236,
    567260,
    527491,
    558583,
    610104,
    624181,
    528669,
    649339,
    499282,
    536255,
    624956,
    475353,
    565505,
    528448,
    487711,
    381311,
    608664,
    655274,
    611071,
    484603,
    537695,
    614468,
    543196,
    533023,
    453488,
    592838,
    425411,
]
len(recomend_ids)

262

In [380]:
df_23 = feat[23].loc[~feat[23]["lo_id"].isin(recomend_ids)]
df_23 = df_23.reset_index(drop=True)

df_23

,lo_id,first_attempt_avg_score,train_count_avg,final_score_avg,train_count_80_avg,oz_done_count,oz_subject_count,oz_percent_avg,course_start,course_50,olymp_start,week_first_login_this_year,wave_login_first,week_first_login_wave,grade
0,528789,76.631700,1.6667,92.657333,0.6667,NaN,NaN,NaN,NaN,NaN,1.0,0,1,2,6
1,587048,NaN,NaN,NaN,NaN,4.0,1.0,80.000000,NaN,NaN,NaN,0,1,23,8
2,593642,NaN,0.0000,NaN,NaN,12.0,2.0,92.307692,NaN,NaN,NaN,0,1,37,7
3,601156,66.259233,1.3333,74.051433,1.0000,14.0,1.0,53.846154,7.0,4.0,1.0,0,1,41,8
4,527197,52.840900,9.5000,52.840900,0.0000,1.0,1.0,100.000000,8.0,6.0,NaN,0,2,42,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12378,657926,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,NaN,NaN,26,0,26,6
12379,657935,12.500000,0.0000,12.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,0,26,6
12380,657937,48.701300,1.0000,48.701300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,0,26,8
12381,648331,21.428600,0.0000,21.428600,NaN,5.0,2.0,50.000000,NaN,NaN,NaN,26,0,26,8


In [382]:
X = df_23[feat_names[:-1]]
# Для 5-ти классников заменим grade на 6
X.loc[X.grade == 5, "grade"] = 6
X

,first_attempt_avg_score,train_count_avg,final_score_avg,train_count_80_avg,oz_done_count,oz_subject_count,oz_percent_avg,course_start,course_50,olymp_start,week_first_login_this_year,wave_login_first,week_first_login_wave,grade
0,76.631700,1.6667,92.657333,0.6667,NaN,NaN,NaN,NaN,NaN,1.0,0,1,2,6
1,NaN,NaN,NaN,NaN,4.0,1.0,80.000000,NaN,NaN,NaN,0,1,23,8
2,NaN,0.0000,NaN,NaN,12.0,2.0,92.307692,NaN,NaN,NaN,0,1,37,7
3,66.259233,1.3333,74.051433,1.0000,14.0,1.0,53.846154,7.0,4.0,1.0,0,1,41,8
4,52.840900,9.5000,52.840900,0.0000,1.0,1.0,100.000000,8.0,6.0,NaN,0,2,42,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12378,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,NaN,NaN,26,0,26,6
12379,12.500000,0.0000,12.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,0,26,6
12380,48.701300,1.0000,48.701300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,0,26,8
12381,21.428600,0.0000,21.428600,NaN,5.0,2.0,50.000000,NaN,NaN,NaN,26,0,26,8


In [383]:
pipe = Pipeline([("preprocessor", preprocessor), ("catboost", cbc_tune)])

proba = pd.Series(pipe.predict_proba(X)[:, 1], name="success_probability")
proba

0        0.403217
1        0.309354
2        0.311617
3        0.428794
4        0.407357
           ...   
12378    0.271882
12379    0.249205
12380    0.289484
12381    0.298222
12382    0.287604
Name: success_probability, Length: 12383, dtype: float64

In [384]:
df_23 = df_23.join(proba).sort_values(by="success_probability", ascending=False)
df_23

,lo_id,first_attempt_avg_score,train_count_avg,final_score_avg,train_count_80_avg,oz_done_count,oz_subject_count,oz_percent_avg,course_start,course_50,olymp_start,week_first_login_this_year,wave_login_first,week_first_login_wave,grade,success_probability
190,525181,87.208633,20.6667,100.0000,0.0,48.0,4.0,50.526316,9.0,7.0,1.0,0,2,30,6,0.472796
922,609024,90.814400,2.6667,100.0000,0.0,83.0,3.0,64.615885,2.0,2.0,NaN,1,0,1,7,0.458512
1617,602863,87.500000,1.0000,100.0000,0.0,54.0,4.0,65.329218,1.0,NaN,NaN,2,1,42,5,0.458303
1077,567361,93.269233,1.0000,100.0000,0.0,53.0,4.0,63.253012,NaN,NaN,NaN,2,2,42,6,0.458236
188,551395,95.000000,1.6667,100.0000,0.0,24.0,4.0,62.500000,5.0,4.0,NaN,0,1,36,5,0.456227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11619,613948,NaN,0.0000,NaN,NaN,0.0,NaN,0.000000,NaN,NaN,NaN,22,0,22,6,0.241139
11571,654801,3.846200,0.0000,3.8462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,0,21,6,0.240680
11555,654714,3.846200,0.0000,3.8462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,0,21,6,0.240680
11469,653824,3.846200,0.0000,3.8462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,0,21,6,0.240680


In [388]:
result = df_23[df_23["grade"] != 8.0].groupby("grade").head(500)
# Увеличим grade на 1, чтобы соответсвовало следующему году
result["grade"] += 1
result

,lo_id,first_attempt_avg_score,train_count_avg,final_score_avg,train_count_80_avg,oz_done_count,oz_subject_count,oz_percent_avg,course_start,course_50,olymp_start,week_first_login_this_year,wave_login_first,week_first_login_wave,grade,success_probability
190,525181,87.208633,20.6667,100.00,0.0,48.0,4.0,50.526316,9.0,7.0,1.0,0,2,30,7,0.472796
922,609024,90.814400,2.6667,100.00,0.0,83.0,3.0,64.615885,2.0,2.0,NaN,1,0,1,8,0.458512
1617,602863,87.500000,1.0000,100.00,0.0,54.0,4.0,65.329218,1.0,NaN,NaN,2,1,42,6,0.458303
1077,567361,93.269233,1.0000,100.00,0.0,53.0,4.0,63.253012,NaN,NaN,NaN,2,2,42,7,0.458236
188,551395,95.000000,1.6667,100.00,0.0,24.0,4.0,62.500000,5.0,4.0,NaN,0,1,36,6,0.456227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5462,614060,68.750000,1.5000,68.75,NaN,NaN,NaN,NaN,2.0,1.0,NaN,8,0,8,6,0.302045
702,551057,30.000000,1.6667,42.50,NaN,2.0,1.0,50.000000,NaN,NaN,NaN,1,1,35,6,0.302029
508,601471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1,1,42,6,0.302023
4920,569619,62.500000,0.0000,62.50,NaN,NaN,NaN,NaN,1.0,1.0,NaN,7,1,4,6,0.301939


In [391]:
result["grade"].value_counts()

grade
7    500
8    500
6    500
Name: count, dtype: int64

## <a id='toc3_9_'></a>[Граница скоринга для 2023-2024 уч. года](#toc0_)

В этом разделе определим границу, по которой мы будем отбирать учеников в будущем году.

Мы хотим выбрать около 20%. 

Алгортм:
1. Посмотрим на оценки модели всех учеников в 2022-2023 учебном году. 
2. Выберем 1000 случайных оценок с заменой
3. Найдем значение 80-перцентиля $p_{80}$
4. Повторим пункты 2-3 1000 раз.
5. Возьмем среднее значение от всех $p_{80}$. Это и будет нашей теоритической границей. 

In [418]:
feat_names.pop()

'na_feature_cnt'

In [430]:
X = feat[23][feat[23].grade != 5][feat_names]
cbc_tune = MyCatboostClassifier()
cbc_tune.load_model("best_tuned_model")

pipe = Pipeline([("preprocessor", preprocessor), ("catboost", cbc_tune)])

proba = pipe.predict_proba(X)[:, 1]
proba

array([0.4032175 , 0.30935353, 0.31161733, ..., 0.2894838 , 0.29822169,
       0.28760413])

In [431]:
def bootstrap_proba_boundary(
    probas: np.ndarray, p: i = 20, sample_size: int = 1000, n_iter: int = 1000
):
    """
    Function takes an array of probabilities and returns a bootstrap mean and std of the top p% scores.

    probas - numpy array of model scores for class 1
    p - top p percents to take
    sample_size - how large a sample to take from probas on each bootstrap round
    n_iter - how many iterations to perform for bootstrap
    """
    percentile_ls = []
    for _ in range(n_iter):
        sample = np.random.choice(proba, sample_size, replace=True)
        percentile_ls.append(np.percentile(sample, 100 - p))
    mean_p = np.mean(percentile_ls)
    std_p = np.std(percentile_ls)
    print(f"Граница ={mean_p: .3f} +-{std_p: .3f}")
    return mean_p, std_p


boundary_mean, boundary_std = bootstrap_proba_boundary(proba)

Граница = 0.359 +- 0.005


Проверим:

In [435]:
len(proba[proba >= boundary_mean]) / len(proba) * 100

19.97537162459319

In [443]:
boundary_mean

0.3587360299263477

Да, реально выбранный процент оказался близким к 20. Также можем оценить сколько из реально рекомендованных попадают в такую выборку:

In [441]:
data = feat[23][feat[23].grade != 5]
n_recomended = claims[23]["recomended"].sum()
print(f"Всего рекомендовано {n_recomended}")
data["success_proba"] = proba
data = data.merge(claims[23], on="lo_id", how="left")
data_boundary = data[data["success_proba"] >= boundary_mean]
recomended_recall = data_boundary["recomended"].sum() / n_recomended
print(f"Процент рекомендованных, попавших в выборку {recomended_recall: .2%}")

Всего рекомендовано 252
Процент рекомендованных, попавших в выборку  85.71%


In [448]:
data.sort_values("success_proba", ascending=False).iloc[:, :-2].to_csv(
    "2022_2023_features_and_scores.csv", index=False
)

## <a id='toc3_10_'></a>[Вывод](#toc0_)

- Выбрана модель CatboostClassifier обученная на данных двух учебных лет с затюнингованными гиперпараметрами в optuna. Она оказалась довольно сильно лучше baseline для базовых гиперпараметров catboost. Eсть надежда, что модель, обученная на последних двух годах, будет более устойчива к колебаниям данных год от года.

- Модель обучалась только на тех учениках, которые подавали заявку в школу. 

- CV показал recall в 40%, при выборе топ 10% процентов по скорингу.

- На всех данных 2022-2023 модель показала recall 85% при выборе 20% всех учеников. Определена граница для отбора учеников в будущем году. 

- Также выполнено предсказание успешности в будущем году (23-24) для 5-7 классов (не показано здесь из-за НДА).